## 6. Optimization

학습 loop
  1. optimizer.zero_grad()를 호출해 모델 parameter의 변화도 재설정
  2. loss.backwards()를 호출해 역전파 실시
  3. optimizer.step()을 호출해 매개변수 조정

In [36]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

**Pre-requiosoe code**

In [37]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_loader = DataLoader(training_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [38]:
model = NeuralNetwork()

**Hyperparameter**

In [39]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

**Optimization implementation**

In [40]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Forward Feed
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch + 1
            print(f'\rloss: {loss:>7f} [{current:>5d}]', end='')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f'\nTest accuracy: {100 * correct :>0.1f}, avg loss: {test_loss:>8f}\n')

**Training**

In [43]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f'Epoch {t + 1}')
    print('-------------------------------')
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 0.203154 [  901]
Test accuracy: 87.9, avg loss: 0.369407

Epoch 2
-------------------------------
loss: 0.196959 [  901]
Test accuracy: 88.3, avg loss: 0.358395

Epoch 3
-------------------------------
loss: 0.189969 [  901]
Test accuracy: 87.4, avg loss: 0.405181

Epoch 4
-------------------------------
loss: 0.127915 [  901]
Test accuracy: 87.8, avg loss: 0.424709

Epoch 5
-------------------------------
loss: 0.156627 [  901]
Test accuracy: 87.7, avg loss: 0.443253

Epoch 6
-------------------------------
loss: 0.119914 [  901]
Test accuracy: 87.8, avg loss: 0.455762

Epoch 7
-------------------------------
loss: 0.099817 [  901]
Test accuracy: 87.9, avg loss: 0.426993

Epoch 8
-------------------------------
loss: 0.100371 [  901]
Test accuracy: 87.9, avg loss: 0.476489

Epoch 9
-------------------------------
loss: 0.134277 [  901]
Test accuracy: 88.1, avg loss: 0.478013

Epoch 10
-------------------------------
loss: 0.139734 [  901]
